# Introduction to the RethinkDB Query Language

The RethinkDB Query Language (ReQL) embeds itself in the programming language that you use to build your application. ReQL is designed as a fluent API, a set of functions that you can chain together to compose queries. Before we build an application, let's start by exploring the query language. 

By default, RethinkDB creates a database named `test`. Let's add a table to the `test` database:

In [2]:
%r r.db("test").table_create("fellowship")

{
    "tables_created": 1, 
    "config_changes": [
        {
            "old_val": null, 
            "new_val": {
                "primary_key": "id", 
                "durability": "hard", 
                "db": "test", 
                "shards": [
                    {
                        "primary_replica": "c77cc04d96c0_6gt", 
                        "replicas": [
                            "c77cc04d96c0_6gt"
                        ]
                    }
                ], 
                "write_acks": "majority", 
                "id": "cd45710e-04fe-4479-a036-0249343d177c", 
                "name": "fellowship"
            }
        }
    ]
}



Now, let's add a set of nine JSON documents to the table:

In [21]:
%r r.table("fellowship").insert([ \
{ "name": "Frodo", "species": "hobbit" }, \
{ "name": "Sam", "species": "hobbit" }, \
{ "name": "Merry", "species": "hobbit" }, \
{ "name": "Pippin", "species": "hobbit" }, \
{ "name": "Gandalf", "species": "istar" }, \
{ "name": "Legolas", "species": "elf" }, \
{ "name": "Gimili", "species": "dwarf" }, \
{ "name": "Aragorn", "species": "human" }, \
{ "name": "Boromir", "species": "human" }])

{
    "errors": 0, 
    "deleted": 0, 
    "generated_keys": [
        "ccf89b51-b3e1-45c5-b59f-a396da284aef", 
        "907a9b8f-6cff-4090-a6b1-7abcc2612953", 
        "449fa0c7-5520-4fd3-9c51-b2a2838d0145", 
        "c4f7915f-d738-4272-aa9a-d9a86b8402c2", 
        "bed70904-464d-4c86-b804-1517de25be83", 
        "7da3a2c8-e1b6-4b69-abb8-5b20b6a05ee7", 
        "1c686a95-bbf3-4bbf-bdcb-bf27d1467bef", 
        "4d5c3f69-811a-4460-964f-2c7db13d13c8", 
        "850f30a8-764c-40f6-8390-03d7bd01cca4"
    ], 
    "unchanged": 0, 
    "skipped": 0, 
    "replaced": 0, 
    "inserted": 9
}



When you run the command above, the database will output an array with the primary keys that it generated for all of the new documents. It will also tell you how many new records it successfully inserted. Now that we have some records in the database, let's try using ReQL's `filter` command to fetch the fellowship's hobbits:

In [23]:
%r r.table("fellowship").filter({"species": "hobbit"})

[
    {
        "species": "hobbit", 
        "id": "ccf89b51-b3e1-45c5-b59f-a396da284aef", 
        "name": "Frodo"
    }, 
    {
        "species": "hobbit", 
        "id": "907a9b8f-6cff-4090-a6b1-7abcc2612953", 
        "name": "Sam"
    }, 
    {
        "species": "hobbit", 
        "id": "449fa0c7-5520-4fd3-9c51-b2a2838d0145", 
        "name": "Merry"
    }, 
    {
        "species": "hobbit", 
        "id": "c4f7915f-d738-4272-aa9a-d9a86b8402c2", 
        "name": "Pippin"
    }
]



The `filter` command retrieves the documents that match the provided boolean expression. In this case, we specifically want documents in which the `species` property is equal to hobbit. You can chain additional commands to the query if you want to perform more operations. For example, you can use the following query to change the value of the `species` property for all hobbits:

In [28]:
%r r.table("fellowship").filter({"species": "hobbit"}) \
                        .update({"species": "halfling"})

{
    "skipped": 0, 
    "deleted": 0, 
    "unchanged": 0, 
    "errors": 0, 
    "replaced": 4, 
    "inserted": 0
}



To selectively remove records from the table, you could use the `delete` command after a `filter` operation. Let's say, for example, that you want to remove Boromir from the fellowship after he is slain by Orcs:

In [29]:
%r r.table("fellowship").filter({"name": "Boromir"}).delete()

{
    "skipped": 0, 
    "deleted": 1, 
    "unchanged": 0, 
    "errors": 0, 
    "replaced": 0, 
    "inserted": 0
}



The `group` command offers a way to combine documents that share the same value on a given property. For example, if we wanted to know how many members of the fellowship correspond with each species, we can use the following ReQL query:

In [30]:
%r r.table("fellowship").group("species").count()

{
    "istar": 1, 
    "halfling": 4, 
    "elf": 1, 
    "dwarf": 1, 
    "human": 1
}



Commands chained after `group` will operate on the contents of the group rather than the entire stream. Consequently, the `count` command in the example above returns the number of items in each group. If you want to perform an operation on the total output rather than the individual groups, you can use the `ungroup` command. For example, we could use the following query if we wanted to know how many total species participated in the fellowship:

In [31]:
%r r.table("fellowship").group("species").ungroup().count()

5



ReQL has a wide range of other commands. It even has a built-in `http` command that you can use to fetch data from public web APIs. In the following example, we use the `http` command to fetch the current posts from a popular subreddit. The full query retrieves the posts, orders them by score, and then displays several properties from the top three entries:

In [34]:
%r r.http("http://www.reddit.com/r/aww.json")["data"]["children"]["data"] \
    .order_by(r.desc("score")).limit(3).pluck("score", "title", "url")

[
    {
        "url": "http://i.imgur.com/daXYEqq.jpg", 
        "score": 4974, 
        "title": "The second male calico the vet has seen in over 54 years of practice."
    }, 
    {
        "url": "http://imgur.com/87xUo9g", 
        "score": 4864, 
        "title": "Are we there yet"
    }, 
    {
        "url": "http://i.imgur.com/htZeGHn.jpg", 
        "score": 4753, 
        "title": "Selby likes to be in control."
    }
]



As you can see, ReQL is very useful for many kinds of ad hoc data analysis. You can use it to slice and dice complex JSON data structures in a number of interesting ways. If you'd like to learn more about ReQL, you can refer to the [API reference documentation](http://rethinkdb.com/api/python/), the [ReQL introduction](http://rethinkdb.com/docs/introduction-to-reql/) on the RethinkDB website, or the [RethinkDB cookbook](http://rethinkdb.com/docs/cookbook/python/).